In [9]:
import numpy as np
import pickle as pkl
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
import warnings
warnings.filterwarnings('ignore')

C:\Users\samth\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


ModuleNotFoundError: No module named 'sklearn'

In [ ]:
df=pd.read_csv("Google stocks.csv")
df.columns = ['ds', 'y']
df.shape

(5179, 2)

In [ ]:
quantiles = [0.015, 0.985]

params = {
    "n_lags": 24,
    "n_forecasts": 7,
    "n_changepoints": 20,
    "learning_rate": 0.01,
    "ar_layers": [32, 16, 16, 32],
    "epochs": 50,
    "batch_size": 64,
    "quantiles": quantiles,
}


m = NeuralProphet(**params)
m.set_plotting_backend("plotly-static")
set_log_level("ERROR")

df_train, df_test = m.split_df(df, valid_p=0.1, local_split=True)
print(f"Train shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")

Train shape: (4831, 2)
Test shape: (563, 2)


In [ ]:
from tensorflow.keras.models import load_model
lstm_model = load_model("model_store/lstm_goog.keras")
lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 75)                23100     
                                                                 
 dense (Dense)               (None, 1)                 76        
                                                                 
Total params: 23176 (90.53 KB)
Trainable params: 23176 (90.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
df_train.shape,df_test.shape

((4831, 2), (563, 2))

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

with open("model_store/best_order_goog.pkl", "rb") as f:
    loaded_order = pkl.load(f)

print("Loaded best order:", loaded_order)

Loaded best order: (1, 1, 1)


In [ ]:
from hmmlearn import hmm

with open("model_store/opt_no_states_goog.pkl", "rb") as f:
    opt_states = pkl.load(f)

print(f'Loaded optimum states : {opt_states}')

Loaded optimum states : 6


## Dynamic Ensemble

In [ ]:
from hmmlearn import hmm
from statsmodels.tsa.arima.model import ARIMA
from utils import softmax_weighting, get_mape_errors, get_mse_errors, get_mae_errors, get_rmse_errors

def dynamic_ensemble_prediction(train, test):
    train_hmm = train.reshape(-1,1)
    train_hmm = train_hmm[1:]-train_hmm[:train_hmm.shape[0]-1]
    test_hmm = test.reshape(-1,1)
    test_hmm = test_hmm[1:]-test_hmm[:test_hmm.shape[0]-1]
    hmm_history = train_hmm
    history = np.array(train)

    predictions = []
    truth_values = []
    lstm_preds = []
    hmm_preds = []
    arima_preds = []

    for i in range(len(test)):
        print(f'{i+1}/{len(test)}')
        truth_values.append(test[i])
        # LSTM
        lstm_pred = lstm_model.predict(history[-24:].reshape(1,24))[0][0]
        lstm_preds.append(lstm_pred)
        # ARIMA
        arima_model = ARIMA(history, order=loaded_order)
        arima_fit = arima_model.fit()
        arima_pred = arima_fit.forecast(steps=1)[0]
        arima_preds.append(arima_pred)
        # HMM
        hmm_model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=0.0001, n_iter=100)
        hmm_model.fit(hmm_history)
        hidden_states = hmm_model.predict(hmm_history)
        last_hidden_state = hidden_states[-1]
        next_state_probs = hmm_model.transmat_[last_hidden_state]
        predicted_state = np.argmax(next_state_probs)
        predicted_change = hmm_model.means_[predicted_state][0]
        hmm_pred = history[-1]+predicted_change
        hmm_preds.append(hmm_pred)

        #Error Measurement
        arima_error = get_mae_errors(arima_preds, truth_values)
        hmm_error = get_mae_errors(hmm_preds, truth_values)
        lstm_error = get_mae_errors(lstm_preds, truth_values) 

        weights = softmax_weighting(arima_error, lstm_error, hmm_error, gamma = 1) # Weighting algorithm

        predictions.append(weights[0]*arima_pred + weights[1]*lstm_pred + weights[2]*hmm_pred)
        history = np.append(history,test[i])
        
        if i != len(test)-1:
            hmm_history = np.append(hmm_history,test_hmm[i]).reshape(-1,1)
    
    return predictions, arima_preds, hmm_preds, lstm_preds
        

In [ ]:
de_preds, arima_preds, hmm_preds, lstm_preds = dynamic_ensemble_prediction(np.array(df_train.y), np.array(df_test.y))

1/563
1/1 [==============================] - 0s 448ms/step
2/563
1/1 [==============================] - 0s 28ms/step
3/563
1/1 [==============================] - 0s 24ms/step
4/563
1/1 [==============================] - 0s 23ms/step
5/563
1/1 [==============================] - 0s 26ms/step
6/563
1/1 [==============================] - 0s 23ms/step
7/563
1/1 [==============================] - 0s 23ms/step
8/563
1/1 [==============================] - 0s 21ms/step


KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import math
print('Dynamic Ensemble')
print(f'R2 Score : {r2_score(df_test.y,de_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,de_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,de_preds)}')
print('ARIMA')
print(f'R2 Score : {r2_score(df_test.y,arima_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,arima_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,arima_preds)}')
print('HMM')
print(f'R2 Score : {r2_score(df_test.y,hmm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,hmm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,hmm_preds)}')
print('LSTM')
print(f'R2 Score : {r2_score(df_test.y,lstm_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y, lstm_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,lstm_preds)}')

Dynamic Ensemble
R2 Score : 0.9937881129564559
RMSE : 2.2421729678057347
MAE : 1.3756846354307894
ARIMA
R2 Score : 0.9910304260031926
RMSE : 2.6942832170066633
MAE : 1.8942667597364462
HMM
R2 Score : 0.9911222841829961
RMSE : 2.680451519737119
MAE : 1.8819705384178416
LSTM
R2 Score : 0.979177385275828
RMSE : 4.105111608708665
MAE : 3.1427208678548757


In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y=df_test['y'], mode='lines', name='Real Data'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_preds, mode='lines', name='Proposed Method'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_preds, mode='lines', name='ARIMA'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_preds, mode='lines', name='HMM'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_preds, mode='lines', name='LSTM'))
fig.add_trace(go.Scatter(x=df_train['ds'], y=df_train['y'], mode='lines', name='Training'))
fig.show()

In [ ]:
# Getting Errors for each prediction of each method
test_df_list = list(df_test['y'])
arima_error = []
hmm_error = []
lstm_error = []
de_error = []
for i in range(len(test_df_list)):
    arima_error.append(mean_absolute_error([test_df_list[i]],[arima_preds[i]]))
    hmm_error.append(mean_absolute_error([test_df_list[i]],[hmm_preds[i]]))
    lstm_error.append(mean_absolute_error([test_df_list[i]],[lstm_preds[i]]))
    de_error.append(mean_absolute_error([test_df_list[i]],[de_preds[i]]))

In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with GOOG Error comparison")
fig.add_trace(go.Scatter(x=df_test['ds'], y=arima_error, mode='lines', name='ARIMA Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=hmm_error, mode='lines', name='HMM Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=lstm_error, mode='lines', name='LSTM Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

## Other Ensembles

### Ensemble Mean

In [ ]:
ensemble_mean_preds = []
ensemble_mean_errors = []
for i in range(len(arima_preds)):
    ensemble_mean_preds.append((arima_preds[i]+hmm_preds[i]+lstm_preds[i])/3)
    ensemble_mean_errors.append(mean_absolute_error([ensemble_mean_preds[i]],[test_df_list[i]]))

print('Ensemble Mean')
print(f'R2 Score : {r2_score(df_test.y,ensemble_mean_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,ensemble_mean_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,ensemble_mean_preds)}')

Ensemble Mean
R2 Score : 0.9899222007843909
RMSE : 2.8558815322315594
MAE : 1.9796895347207424


In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Mean Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = ensemble_mean_errors, mode='lines', name='Mean Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Best model in hindsight

In [ ]:
bmh_preds = []
bmh_errors = []
mape = [mean_absolute_percentage_error(arima_preds, df_test['y']), mean_absolute_percentage_error(hmm_preds, df_test['y']), mean_absolute_percentage_error(lstm_preds, df_test['y'])]
bmh_preds = [arima_preds,hmm_preds,lstm_preds][np.argmin(mape)]

for i in range(len(test_df_list)):
    bmh_errors.append(mean_absolute_error([bmh_preds[i]],[test_df_list[i]]))

print('Best Model in Hindsight')
print(f'R2 Score : {r2_score(df_test.y,bmh_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,bmh_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,bmh_preds)}')

Best Model in Hindsight
R2 Score : 0.9911222841829961
RMSE : 2.680451519737119
MAE : 1.8819705384178416


In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with BMH Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = bmh_errors, mode='lines', name='BMH Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Ridge Ensemble

In [ ]:
import numpy as np
from sklearn.linear_model import Ridge

def ridge_ensemble(forecasts, actuals, alpha=1.0):
    X = np.array(forecasts).T  
    y = np.array(actuals)       

    ridge = Ridge(alpha=alpha, fit_intercept=False)
    ridge.fit(X, y)
    
    weights = ridge.coef_
    ensemble_prediction = np.dot(X, weights)

    return {"weights": weights, "ensemble_prediction": ensemble_prediction}

In [ ]:
forecasts = [arima_preds, hmm_preds, lstm_preds]
ridge_output = ridge_ensemble(forecasts, test_df_list)
ridge_preds = ridge_output['ensemble_prediction']
ridge_errors = []
for i in range(len(test_df_list)):
    ridge_errors.append(mean_absolute_error([ridge_preds[i]],[test_df_list[i]]))

print('Ridge Ensemble')
print(f'R2 Score : {r2_score(df_test.y,ridge_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,ridge_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,ridge_preds)}')

Ridge Ensemble
R2 Score : 0.9911362088771486
RMSE : 2.678348551151759
MAE : 1.8772231112721072


In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Ridge Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = ridge_errors, mode='lines', name='Ridge Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Exp3 Ensemble

In [ ]:
class Exp3Ensemble:
    def __init__(self, num_models, window_size=1):
        self.num_models = num_models
        self.window_size = window_size
        self.regret = np.zeros(num_models)  # Initial regret for all models
        self.weights = np.ones(num_models) / num_models  # Initial equal weights

    def update(self, forecasts, actuals, t):
        if t < self.window_size:
            return 

        for i in range(self.num_models):
            self.regret[i] = np.sum([(actuals[s] - forecasts[i][s]) ** 2 for s in range(t - self.window_size, t)])

        eta_t = np.sqrt(8 * np.log(self.num_models) / self.window_size)
        new_weights = np.exp(-eta_t * self.regret)
        self.weights = new_weights / np.sum(new_weights)  # Normalize and update weights

    def get_ensemble_prediction(self, forecasts):
        return np.dot(self.weights, forecasts)

num_models = 3
window_size = 10
exp3_ensemble = Exp3Ensemble(num_models, window_size)

forecasts = [arima_preds, hmm_preds, lstm_preds]

exp3_preds = []
for t in range(len(test_df_list)):
    exp3_ensemble.update(forecasts, test_df_list, t)
    exp3_preds.append(exp3_ensemble.get_ensemble_prediction([[model[t]] for model in forecasts]))

exp3_errors = []
for i in range(len(test_df_list)):
    exp3_errors.append(mean_absolute_error([exp3_preds[i]],[test_df_list[i]]))

In [ ]:
print('Exp3 Ensemble')
print(f'R2 Score : {r2_score(df_test.y,exp3_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,exp3_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,exp3_preds)}')

Exp3 Ensemble
R2 Score : 0.9907821445677876
RMSE : 2.731318105711834
MAE : 1.9364216101340226


In [ ]:
fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Exp3 Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = exp3_errors, mode='lines', name='Exp3 Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

###  Passive Aggressive

In [ ]:
import numpy as np

class PassiveAggressiveForecaster:
    def __init__(self, num_models, epsilon=2):
        self.epsilon = epsilon  # Margin parameter
        self.beta = np.ones(num_models) / num_models  # Initialize weights equally

    def update_weights(self, X_t, y_t):
        X_t = np.array(X_t)  # Ensure it's a NumPy array
        y_t_pred = np.dot(X_t, self.beta)  # Weighted prediction

        loss = abs(y_t_pred - y_t) - self.epsilon
        tau_t = max(0, loss) / (np.linalg.norm(X_t) ** 2 + 1e-8)  # Small term to avoid division by zero

        self.beta += np.sign(y_t_pred - y_t) * tau_t * X_t

        self.beta = np.maximum(self.beta, 0)  # Ensure non-negative weights
        self.beta /= np.sum(self.beta)  # Normalize weights

    def predict(self, X_t):
        return np.dot(X_t, self.beta)


num_models = 3

# Hyper parameter tuning
maxr2 = 0
maxep=-1
for eps in np.arange(0.1,50,0.2):
    forecaster = PassiveAggressiveForecaster(num_models,eps)
    pa_preds = []
    for t in range(len(test_df_list)):
        X_t = [model[t] for model in forecasts] 
        y_t = test_df_list[t]  
        pa_preds.append(forecaster.predict(X_t))
        forecaster.update_weights(X_t, y_t)
    
    if maxr2 < r2_score(df_test.y,pa_preds):
        maxep = eps
        maxr2 = r2_score(df_test.y,pa_preds)
    
forecaster = PassiveAggressiveForecaster(num_models,maxep)
pa_preds = []
for t in range(len(test_df_list)):
    X_t = [model[t] for model in forecasts]  # Model predictions at time t
    y_t = test_df_list[t]  # Actual value at time t

    # Predict and update weights
    pa_preds.append(forecaster.predict(X_t))
    forecaster.update_weights(X_t, y_t)

In [ ]:
print('Passive Aggressive Ensemble')
print(f'R2 Score : {r2_score(df_test.y,pa_preds)}')
print(f'RMSE : {math.sqrt(mean_squared_error(df_test.y,pa_preds))}')
print(f'MAE : {mean_absolute_error(df_test.y,pa_preds)}')

Passive Aggressive Ensemble
R2 Score : 0.9899222007843909
RMSE : 2.855881532231557
MAE : 1.9796895347207417


In [ ]:
pa_errors = []
for i in range(len(test_df_list)):
    pa_errors.append(mean_absolute_error([pa_preds[i]],[test_df_list[i]]))

fig = go.Figure()
fig.update_layout(title="Dynamic Ensemble with Passive Aggressive Ensemble MAE comparison - GOOG")
fig.add_trace(go.Scatter(x=df_test['ds'], y = pa_errors, mode='lines', name='Passive Aggressive Ensemble Error'))
fig.add_trace(go.Scatter(x=df_test['ds'], y=de_error, mode='lines', name='Proposed Method Error'))
fig.show()

### Adaptive Robust Optimization

In [ ]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import time

def get_X_Z_y(X, y, T):
    n, p = X.shape
    #T past time steps * p features + T targets + intercept term
    Z = np.ones((n-T, T*p+T+1))
    for i in range(T, n):
        for t in range(T):
            Z[i-T,p*t:p*(t+1)] = X[i-t-1]
        Z[i-T, p*T:-1] = y[i-T:i]
    return X[T:], Z, y[T:]

def adaptive_ridge_regression_standard(X, y, rho_beta0, rho_V0, T):

    # Create model
    m = ConcreteModel()
    X, Z, y = get_X_Z_y(X, y, T)

    N, P = X.shape
    # Add variables
    m.beta0 = Var(range(P + 1))
    m.V0 = Var(range(P + 1), range(T * P + T + 1))
    m.t = Var(domain=NonNegativeReals)
    # Add objective
    print(Z.shape)
    m.obj = Objective(sense=minimize, expr=m.t + rho_beta0 * sum(pow(m.beta0[j], 2) for j in range(P))
                                           + rho_V0 * sum(
        pow(m.V0[i, j], 2) for i in range(P) for j in range(T * P + T + 1))
                      )

    m.quadratic = Constraint(expr=m.t >= 1 / N * sum(
        pow(y[i] -
            sum(
                X[i, j] *
                (m.beta0[j]
                 + sum(m.V0[j, l] * Z[i, l] for l in range(T * P + T + 1))
                 # +np.matmul(m.V0,Z[i].reshape(97,1))[j]
                 )
                for j in range(P))
            - m.beta0[P] - sum(m.V0[P, l] * Z[i, l] for l in range(T * P + T + 1))
            , 2)
        for i in range(N)))

    solver = SolverFactory('glpk')  # 'ipopt', executable=executable)
    start_time = time.time()
    ## tee=True enables solver output
    results = solver.solve(m, tee=True)
    print("--- %s seconds ---" % (time.time() - start_time))
    # results = solver.solve(m, tee=False)
    V0 = np.array([[m.V0[j, l].value for l in range(P * T + T + 1)] for j in range(P + 1)])
    return m, np.array([m.beta0[j].value for j in range(P + 1)]), V0

In [ ]:
import os
import pyomo.environ as pyo

# Add GLPK directory manually
os.environ["PATH"] += os.pathsep + "C:/glpk-4.65/w64"

solver = pyo.SolverFactory("glpk")
print(solver.available())  # Should print True

True


In [ ]:
out = adaptive_ridge_regression_standard(np.array(forecasts), np.array(test_df_list), 1, 1, 1)

NameError: name 'forecasts' is not defined

In [161]:
def compute_cvar(errors, risk=0.05):
    var_threshold = np.percentile(errors, (risk) * 100)  
    cvar = errors[errors <= var_threshold].mean() 
    return cvar

In [126]:
compute_cvar(np.array(de_error),0.05)

0.040890661403625676